# Описание проекта

Необходимо проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

Имеются данные о поведении клиентов, которые уже перешли на эти тарифы.<br>
Нужно построить модель, которая выберет подходящий тариф.


# Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

#### Известно:
-    сalls — количество звонков,
-    minutes — суммарная длительность звонков в минутах,
-    messages — количество sms-сообщений,
-    mb_used — израсходованный интернет-трафик в Мб,
-    is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 1. Откроем и изучим файл

In [1]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# # будем отображать графики прямо в jupyter'e
# %matplotlib inline
# # #графики в svg выглядят более четкими
# %config InlineBackend.figure_format = 'svg'

In [2]:
import pandas as pd
import numpy as np

In [3]:
import sys
print(sys.version)

# !pip install scikit-learn==0.21.2

import pandas as pd

import joblib
print('The joblib version is {}.'.format(joblib.__version__))

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

# _1_
# Классификатор дерева решений
from sklearn.tree import DecisionTreeClassifier
#_2_
# Классификатор случайного леса
from sklearn.ensemble import RandomForestClassifier
#_3_
# Линейная модель
from sklearn.linear_model import LogisticRegression
 
# Оценка правильности
from sklearn.metrics import accuracy_score
# Разделение на тренировочную и валидационную выборку
from sklearn.model_selection import train_test_split
# Средняя квадратичная ошибка
from sklearn.metrics import mean_squared_error
# Оценка качества модели
from sklearn.metrics import classification_report
# Перебор параметров
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

# #_4_ целевой признак не позволяет использовать регрессию
# # Решающее дерево для регрессии 
# from sklearn.tree import DecisionTreeRegressor
# #_5_
# # Случайный лес для регрессии 
# from sklearn.ensemble import RandomForestRegressor
# #_6_
# # Линейная регрессия называется 
# from sklearn.linear_model import LinearRegression

3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:57:50) 
[GCC 7.5.0]
The joblib version is 0.16.0.
The scikit-learn version is 0.21.2.


In [4]:
# Для оформления 
bold_start = '\033[1m'
yellow = '\033[33m'
red = '\033[31m'
violet = '\033[35m'
blue = '\033[34m'
green = '\033[32m'
green_background = '\033[42m'
red_background = '\033[41m'
bold_end   = '\033[0m'

In [5]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [6]:
display(df.head())
display(df.info())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

In [7]:
df['calls'] = df.calls.astype('Int64')
df['messages'] = df.messages.astype('Int64')

## 2. Разобьем данные на выборки

***GridSearchCV*** производит кросс валидацию

Кроме этого в scikit-learn есть класс ***TimeSeriesSplit***, но он разбивает только на train-test, валидацию все равно делать самим

```python

# 70% данных на обучение, 15% в тестах и 15 % в валидации.

# но такой споб нам не подходит из за отсутсвия random_state

probs = np.random.rand(len(df))
training_mask = probs < 0.7
test_mask = (probs>=0.7) & (probs < 0.85)
validatoin_mask = probs >= 0.85


df_training = df[training_mask]
df_test = df[test_mask]
df_validation = df[validatoin_mask]

# display(df_training)
# display(df_test)
# display(df_validation)

```

80% данных на обучение, 10% в тестах и 10% в валидации

Некрасиво, переделаем

```python

#######################################################################
# Разделим данные на обучающую, валидационную и тренировочную выборки #
#######################################################################

df_divisible, df_valid = train_test_split(df, test_size=0.20, random_state=12345)
df_train, df_test = train_test_split(df_divisible, test_size=0.25, random_state=12345)

display(df_train.shape)
display(df_valid.shape)
display(df_test.shape) 


#########################################################
# Создадим переменные для признаков и целевого признака #
# features (англ. «признаки») — запишем в неё признаки  #
# target (англ. «цель») — целевой признак               #
#########################################################

# train
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

# valid
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

# test
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

```

 <a name="contents_1"></a>
### ! [Описание причины удаления столбца **calls** ](#stage_1)
P.S. Нажми

In [8]:
features = df.drop(['is_ultra','calls'], axis = 1)
target = df['is_ultra']

# организуем равномерное распределение значений обучающего столба 
# при разделении с помощью stratify !

features_train, features_val_test, target_train, target_val_test = train_test_split(features, target,
                                                            test_size=0.20, random_state=12345, stratify=target)

features_valid, features_test, target_valid, target_test = train_test_split(features_val_test, target_val_test,
                                                            test_size=0.5, random_state=12345, stratify=target_val_test)
features_train.shape, features_val_test.shape, features_valid.shape, features_test.shape

((2571, 3), (643, 3), (321, 3), (322, 3))

In [9]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

## 3. Исследем модели

In [10]:
# Функция вывода accuracy
def print_accuracy(accuracy_in_model):
    if accuracy_in_model < 0.75:
        print( red, "Accuracy:", accuracy_in_model, bold_end, 
          '\n', bold_start, red_background, "need to update", bold_end)
    else:
        print(green, "Accuracy:", accuracy_in_model, bold_end, 
          '\n', bold_start, green_background, "OK", bold_end)
    print(blue,
          bold_start,
          'Оценка качества модели',
          bold_end, '\n', blue,
          classification_report(target_valid, prediction))   

---

* ### DecisionTreeClassifier

In [11]:
model_DecisionTreeClassifier = DecisionTreeClassifier(random_state=12345)
# Обучение
model_DecisionTreeClassifier.fit(features_train, target_train)
# Предсказание понадобится для оценки модели
prediction = model_DecisionTreeClassifier.predict(features_valid)
# Доля правильных ответов
accuracy = model_DecisionTreeClassifier.score(features_valid, target_valid)
#### 
print_accuracy(accuracy)

 Accuracy: 0.7414330218068536  
   need to update 
  Оценка качества модели  
                precision    recall  f1-score   support

           0       0.82      0.80      0.81       223
           1       0.57      0.60      0.59        98

    accuracy                           0.74       321
   macro avg       0.70      0.70      0.70       321
weighted avg       0.75      0.74      0.74       321



 зададим глубину max_depth=3

In [12]:
model_DecisionTreeClassifier = DecisionTreeClassifier(random_state=12345,
                                                      max_depth=3)
# Обучение
model_DecisionTreeClassifier.fit(features_train, target_train)
# Предсказание
prediction = model_DecisionTreeClassifier.predict(features_valid)
# Доля правильных ответов
accuracy = accuracy_score(target_valid, prediction)
###
print_accuracy(accuracy)

 Accuracy: 0.778816199376947  
   OK 
  Оценка качества модели  
                precision    recall  f1-score   support

           0       0.80      0.91      0.85       223
           1       0.70      0.48      0.57        98

    accuracy                           0.78       321
   macro avg       0.75      0.69      0.71       321
weighted avg       0.77      0.78      0.77       321



Сделаем перебор max_depth

In [13]:
best_score = 0
best_depth = 0  
for depth in range(2,11):
# Cоздадим модель, указав max_depth=depth 
    model_DecisionTreeClassifier = DecisionTreeClassifier(random_state=12345,
                                                          max_depth=depth)
# Обучение
    model_DecisionTreeClassifier.fit(features_train, target_train)
# Предсказание понадобится для оценки модели
    prediction =  model_DecisionTreeClassifier.predict(features_valid)

    # print("max_depth =", depth, ": ", end='')
    score_model_DecisionTreeClassifier = model_DecisionTreeClassifier.score(features_valid, target_valid)
    # print(accuracy_score(target_test, predictions_test))
    if score_model_DecisionTreeClassifier >= best_score:
        best_score = score_model_DecisionTreeClassifier
        best_depth = depth
print(green,'best max_depth:' , best_depth)
print_accuracy(best_score)

 best max_depth: 6
 Accuracy: 0.8068535825545171  
   OK 
  Оценка качества модели  
                precision    recall  f1-score   support

           0       0.80      0.93      0.86       223
           1       0.74      0.47      0.58        98

    accuracy                           0.79       321
   macro avg       0.77      0.70      0.72       321
weighted avg       0.78      0.79      0.77       321



---

* ###  RandomForestClassifier

In [14]:
best_score = 0
best_depth = 0                                      
for depth in range(2,11):
# Cоздадим модель, указав max_depth=depth=depth     
    model_RandomForestClassifier = RandomForestClassifier(random_state=12345,
                                                          n_estimators=100,
                                                          max_depth=depth)
# Обучение    
    model_RandomForestClassifier.fit(features_train, target_train)
# Предсказание понадобится для оценки модели
    prediction =  model_RandomForestClassifier.predict(features_valid)
    
    # print("max_depth =", depth, ": ", end='')
    score_model_RandomForestClassifier = model_RandomForestClassifier.score(features_valid, target_valid)
    # print(accuracy_score_model_RandomForestClassifier)
    if score_model_RandomForestClassifier > best_score:                             
        best_score = score_model_RandomForestClassifier 
        best_depth = depth 
print(green,'best max_depth:' , best_depth)
print_accuracy(best_score)

 best max_depth: 8
 Accuracy: 0.8317757009345794  
   OK 
  Оценка качества модели  
                precision    recall  f1-score   support

           0       0.82      0.96      0.89       223
           1       0.87      0.53      0.66        98

    accuracy                           0.83       321
   macro avg       0.85      0.75      0.77       321
weighted avg       0.84      0.83      0.82       321



Вот и наш идеал в деревьях

In [15]:
best_model = RandomForestClassifier(random_state=12345,n_estimators=100, max_depth=best_depth)
best_model.fit(features_train, target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

чтобы избавиться от вложенных циклов по перебору параметров, подключим GridSearchCV

In [16]:
# Словарь параметров
param_grid = {'max_depth': [deep for deep in range(2,11)],
             'n_estimators': [20, 50, 100],
             'random_state': [12345]}
gsCV = GridSearchCV(RandomForestClassifier(), param_grid = param_grid,)

In [17]:
features_GCV = pd.concat([features_train, features_valid]).reset_index(drop=True)
target_GCV = pd.concat([pd.Series(target_train), target_valid]).reset_index(drop=True)
features_GCV,target_upsampling_GCV = shuffle(features_GCV, target_GCV,
                                                        random_state=12345)

In [18]:
# На вход лучше подавать данных побольше, поэтому можно взять первую разбивку
# train + valid
gsCV.fit(features_GCV, target_GCV)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [19]:
gsCV.best_params_

{'max_depth': 3, 'n_estimators': 50, 'random_state': 12345}

In [20]:
print_accuracy(gsCV.best_score_)

 Accuracy: 0.6939834024896265  
   need to update 
  Оценка качества модели  
                precision    recall  f1-score   support

           0       0.82      0.96      0.89       223
           1       0.87      0.53      0.66        98

    accuracy                           0.83       321
   macro avg       0.85      0.75      0.77       321
weighted avg       0.84      0.83      0.82       321



In [21]:
prediction = gsCV.predict(features_valid)
print(classification_report(target_valid, prediction))

              precision    recall  f1-score   support

           0       0.69      1.00      0.82       223
           1       0.00      0.00      0.00        98

    accuracy                           0.69       321
   macro avg       0.35      0.50      0.41       321
weighted avg       0.48      0.69      0.57       321



In [22]:
print_accuracy(gsCV.score(features_GCV, target_GCV))

 Accuracy: 0.6936376210235131  
   need to update 
  Оценка качества модели  
                precision    recall  f1-score   support

           0       0.69      1.00      0.82       223
           1       0.00      0.00      0.00        98

    accuracy                           0.69       321
   macro avg       0.35      0.50      0.41       321
weighted avg       0.48      0.69      0.57       321



---

* ###  LogisticRegression

In [23]:
best_score = 0
best_depth = 0  
for depth in range(2, 100): #Комментарий наставника: эту переменную
# Cоздадим модель, указав max_iter=depth 
    model_LogisticRegression = LogisticRegression(random_state=12345,
                                                  max_iter=depth)
# Обучение
    model_LogisticRegression.fit(features_train, target_train)
# Предсказание понадобится для оценки модели
    prediction =  model_LogisticRegression.predict(features_valid)

    # print("max_depth =", depth, ": ", end='')
    accuracy_score_model_LogisticRegression = model_LogisticRegression.score(features_valid, target_valid)
    # print(accuracy_score_model_LogisticRegression)
    if accuracy_score_model_LogisticRegression > best_score:
        best_score = accuracy_score_model_LogisticRegression
        best_depth = depth
print(green,'best max_ite:' , best_depth)
print_accuracy(best_score)

 best max_ite: 8
 Accuracy: 0.7165109034267912  
   need to update 
  Оценка качества модели  
                precision    recall  f1-score   support

           0       0.72      0.98      0.83       223
           1       0.71      0.12      0.21        98

    accuracy                           0.72       321
   macro avg       0.71      0.55      0.52       321
weighted avg       0.71      0.72      0.64       321



In [24]:
param_grid = {'penalty': ['l1','l2'],
             'C': [0.5, 0.1],
             'max_iter': [deep for deep in range(2,100)]}
gsCV = GridSearchCV(LogisticRegression(), param_grid = param_grid)

In [25]:
gsCV.fit(features_train, target_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.5, 0.1],
                         'max_iter': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                      14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                                      24, 25, 26, 27, 28, 29, 30, 31, ...],
                         'penalty': ['l1', 'l2']},
             

In [26]:
gsCV.best_params_

{'C': 0.5, 'max_iter': 16, 'penalty': 'l1'}

In [27]:
print_accuracy(gsCV.best_score_)

 Accuracy: 0.750291715285881  
   OK 
  Оценка качества модели  
                precision    recall  f1-score   support

           0       0.72      0.98      0.83       223
           1       0.71      0.12      0.21        98

    accuracy                           0.72       321
   macro avg       0.71      0.55      0.52       321
weighted avg       0.71      0.72      0.64       321



In [28]:
best_score = gsCV.score(features_valid, target_valid)
print_accuracy(best_score)

 Accuracy: 0.7507788161993769  
   OK 
  Оценка качества модели  
                precision    recall  f1-score   support

           0       0.72      0.98      0.83       223
           1       0.71      0.12      0.21        98

    accuracy                           0.72       321
   macro avg       0.71      0.55      0.52       321
weighted avg       0.71      0.72      0.64       321



Доработали, но все равно это не самая лучшая модель

Определим на сколько наша модель рандомная

In [29]:
from sklearn.dummy import DummyClassifier

In [30]:
dc = DummyClassifier(strategy = 'most_frequent', random_state = 12345)
dc.fit(features_train, target_train)

DummyClassifier(constant=None, random_state=12345, strategy='most_frequent')

In [31]:
predict_dc = dc.predict(features_valid)

In [32]:
accuracy_score(predict_dc, target_valid)

0.6947040498442367

<a name="stage_1"></a>
## [Описание причины удаления столбца **calls** &darr;](#stage_1)
[Наверх в начало](#contents_1)


&darr; &darr; &darr; &darr; &darr; &darr; &darr; &darr; 

В данном случае логистическая модель предсказывает хуже чем деревья

посмотрим корреляции факторов для исключения плохой обучаемости в случае логистической регрессии

In [33]:
df[df.columns].corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


большая зависимость между количеством звонков и минутами, для деревьев это не страшно, а вот логистическя регрессия от этого страдает сильно.
вернемся в начало и удалим столбец с колличеством звонков 

---

## 4. Проверка модели на тестовой выборке

In [34]:
prediction =  model_RandomForestClassifier.predict(features_test)

score_best_model = best_model.score(features_test, target_test)
print(green, "score:", score_best_model, bold_end,
      '\n', bold_start, green_background, "OK", bold_end)
print(blue,
          bold_start,
          'Оценка качества модели',
          bold_end, '\n', blue,
          classification_report(target_test, prediction))

 score: 0.8074534161490683  
   OK 
  Оценка качества модели  
                precision    recall  f1-score   support

           0       0.81      0.95      0.87       223
           1       0.82      0.49      0.62        99

    accuracy                           0.81       322
   macro avg       0.81      0.72      0.75       322
weighted avg       0.81      0.81      0.79       322

